In [10]:
import subprocess

Starting: Uninstalling PyMuPDF
Completed: Uninstalling PyMuPDF
Starting: Uninstalling fitz
Completed: Uninstalling fitz
Starting: Installing PyMuPDF
Completed: Installing PyMuPDF
PyMuPDF (fitz) installed successfully. Module location: C:\Users\holme\anaconda3\Lib\site-packages\fitz\__init__.py


In [11]:
import fitz
print(fitz.__file__)


C:\Users\holme\anaconda3\Lib\site-packages\fitz\__init__.py


In [12]:
import os
import re
import fitz

# Define the path to your documents
docs_path = r'D:\Data Analysis ML AI\irish_law'


# Function to extract text from each PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Function to clean up each document
def clean_text(text):
    # Remove common non-legal content like headers, footers, and page numbers
    text = re.sub(r'\n\d+\n', ' ', text)  # Remove page numbers
    text = re.sub(r'\s+', ' ', text)       # Remove extra whitespace
    text = re.sub(r'HEADER_TEXT|FOOTER_TEXT', '', text)  # Replace with any common header/footer text patterns

    # Remove other unwanted content if there are any known phrases or symbols (adjust as needed)
    non_legal_phrases = ["Confidential", "Draft Copy"]
    for phrase in non_legal_phrases:
        text = text.replace(phrase, '')

    return text.strip()

# Apply extraction and cleaning function to all PDF documents in the folder
cleaned_docs = {}
for filename in os.listdir(docs_path):
    if filename.endswith('.pdf'):  # Check for PDF files
        pdf_path = os.path.join(docs_path, filename)
        print(f"Processing file: {filename}")
        
        # Extract and clean text
        raw_text = extract_text_from_pdf(pdf_path)
        cleaned_text = clean_text(raw_text)
        cleaned_docs[filename] = cleaned_text

# Check the output
if cleaned_docs:
    for filename, text in cleaned_docs.items():
        print(f"Document: {filename}")
        print("Sample cleaned text:", text[:200], "\n")  # Preview first 200 characters
else:
    print("No documents were processed or the documents are empty.")


Processing file: Policing, Security and Community Safety Act 2024.pdf
Processing file: Digital Services Act 2024.pdf
Processing file: Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf
Processing file: Coroners (Amendment) Act 2024.pdf
Processing file: Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf
Processing file: Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf
Processing file: Local Government (Mayor of Limerick) and Miscellaneous Provisions Act 2024.pdf
Processing file: Criminal Justice (Engagement of Children in Criminal Activity) Act 2024.pdf
Processing file: European Arrest Warrant (Amendment) Act 2024.pdf
Processing file: Road Traffic Act 2024.pdf
Processing file: Gas (Amendment) and Miscellaneous Provisions Act 2024.pdf
Processing file: Court Proceedings (Delays) Act 2024.pdf
Processing file: Health (Termination of Pregnancy Services) (Safe Acce

In [13]:
import os
import re
import fitz  # PyMuPDF
from fuzzywuzzy import fuzz

# Define the path to your documents
docs_path = r'D:\Data Analysis ML AI\irish_law'

# Define keywords for each category
keywords_dict = {
    "Employment Law": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal",
        "Employment", "Redundancies", "Companies", "Labour", "Workplace", "Wages", "Union", "Employee", 
        "Contract", "Pensions", "Rights", "Protection", "Compensation", "Officer", "Board", "Agreement"
    ],
    "Employee Rights and Benefits": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal"
    ],
    "Labour Relations and Unions": [
        "Collective Bargaining", "Strike", "Industrial Action", "Union Membership", "Labor Dispute", 
        "Union Representation", "Employee Representation"
    ],
    "Workplace Policies": [
        "Harassment", "Discrimination", "Equal Opportunity", "Health and Safety", "Workplace Conduct", 
        "Professional Standards", "Ethics in Employment", "Grievance Procedures"
    ],
    "Employment Contracts and Agreements": [
        "Fixed-Term", "Permanent", "Temporary", "Probationary Period", "Notice Period", "Employment Contract", 
        "Part-Time", "Full-Time", "Independent Contractor"
    ],
    "Employment Law Compliance": [
        "Statutory Requirements", "Regulatory Compliance", "Employment Tribunal", "Disciplinary Action", 
        "Legal Protection", "Welfare", "Fair Treatment"
    ],
    "Financial and Economic Law": [
        "Finance", "Tax", "Economic", "Guarantees", "Funds", "Investment", "Banking", "Money", "Budget", 
        "Revenue", "Assets", "Financial", "Capital", "Insurance", "Fiscal"
    ],
    "Technology and Digital Law": [
        "Digital", "Data", "Privacy", "Cybersecurity", "Online", "Internet", "Information", "Communications", 
        "Telecommunications"
    ],
    "Social Welfare Law": [
        "Social Welfare", "Assistance", "Housing", "Disability", "Unemployment", "Pension", "Family Support", 
        "Public Assistance", "Benefits", "Community"
    ],
    "Research and Innovation Law": [
        "Research", "Innovation", "Agency", "Science", "Development", "Superannuation", "Board", "Funding", 
        "Minister", "Establishment"
    ],
    "Retirement and Pensions Law": [
        "Retirement", "Pension", "Savings", "Enrolment", "Superannuation"
    ],
    "Education and Training Law": [
        "Education", "Training", "Schools", "Teaching", "Apprenticeship", "Learning"
    ],
    "Corporate and Business Law": [
        "Corporate", "Business", "Trade", "Commerce", "Industry", "Company", "Commercial"
    ],
    "Employment and Labour Law": [
        "Employment", "Union", "Agreement", "Officer", "Publications", "Board", "Member", "Wages", "Labour", 
        "Contract", "Workforce", "Employee Benefits", "Labour Standards", "Industrial Relations", "Working Time", 
        "Paternity Leave", "Employee Rights", "Standards in Public Office"
    ],
    "Data and Privacy Law": [
        "Information", "Publications", "Notice", "Statutory", "Scheme", "Regulations", "Directive", 
        "Confidentiality", "Data Retention", "Information Security", "Disclosure", "Data Protection", 
        "Freedom of Information", "Protected Disclosures", "Privacy", "Official Secrets"
    ]
}

# Set similarity threshold
SIMILARITY_THRESHOLD = 80

# Function to extract text from each PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Function to clean up each document
def clean_text(text):
    text = re.sub(r'\n\d+\n', ' ', text)  # Remove page numbers
    text = re.sub(r'\s+', ' ', text)       # Remove extra whitespace
    text = re.sub(r'HEADER_TEXT|FOOTER_TEXT', '', text)  # Replace common header/footer text patterns
    non_legal_phrases = ["Confidential", "Draft Copy"]
    for phrase in non_legal_phrases:
        text = text.replace(phrase, '')
    return text.strip()

# Function to check if text contains phrases similar to any keywords
def contains_similar_phrase(text, categories, threshold=SIMILARITY_THRESHOLD):
    words_in_text = text.split()  # Convert text to a list of words for sliding window
    for category, keywords in categories.items():
        for keyword in keywords:
            # Check if any phrase within the text matches the keyword based on threshold
            for i in range(len(words_in_text) - len(keyword.split()) + 1):
                phrase = " ".join(words_in_text[i:i + len(keyword.split())])
                similarity = fuzz.partial_ratio(keyword.lower(), phrase.lower())
                if similarity >= threshold:
                    return True
    return False

# Apply extraction, cleaning, and fuzzy filtering functions to all PDF documents
cleaned_docs = {}
for filename in os.listdir(docs_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(docs_path, filename)
        print(f"Processing file: {filename}")
        
        # Extract and clean text
        raw_text = extract_text_from_pdf(pdf_path)
        cleaned_text = clean_text(raw_text)
        
        # Check if the document text contains similar phrases
        if contains_similar_phrase(cleaned_text, keywords_dict):
            cleaned_docs[filename] = cleaned_text
            print(f"Document '{filename}' matched and was saved.")
        else:
            print(f"Document '{filename}' did not match any relevant keywords.")

# Final output summary
if cleaned_docs:
    print(f"\nTotal matched documents: {len(cleaned_docs)}")
    for filename, text in cleaned_docs.items():
        print(f"Relevant Document: {filename}")
        print("Sample cleaned text:", text[:200], "\n")  # Preview first 200 characters
else:
    print("No relevant documents were found.")


Processing file: Policing, Security and Community Safety Act 2024.pdf


C:\Users\holme\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Document 'Policing, Security and Community Safety Act 2024.pdf' matched and was saved.
Processing file: Digital Services Act 2024.pdf
Document 'Digital Services Act 2024.pdf' matched and was saved.
Processing file: Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf
Document 'Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf' matched and was saved.
Processing file: Coroners (Amendment) Act 2024.pdf
Document 'Coroners (Amendment) Act 2024.pdf' matched and was saved.
Processing file: Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf
Document 'Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf' matched and was saved.
Processing file: Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf
Document 'Social Welfare and Civil Law (Miscellaneous Provisions) Act 2

In [14]:
# Initialize a dictionary to count matches per category
category_summary = {category: 0 for category in keywords_dict.keys()}


In [15]:
from collections import defaultdict
import os
import fitz  # PyMuPDF
from fuzzywuzzy import fuzz

# Use the same keywords_dict and SIMILARITY_THRESHOLD as before

# Define the path to your documents
docs_path = r'D:\Data Analysis ML AI\irish_law'

# Function to extract text from each PDF (reuse from the first code)
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Function to clean text (reuse from the first code)
def clean_text(text):
    text = re.sub(r'\n\d+\n', ' ', text)  # Remove page numbers
    text = re.sub(r'\s+', ' ', text)       # Remove extra whitespace
    text = re.sub(r'HEADER_TEXT|FOOTER_TEXT', '', text)  # Replace common header/footer text patterns
    non_legal_phrases = ["Confidential", "Draft Copy"]
    for phrase in non_legal_phrases:
        text = text.replace(phrase, '')
    return text.strip()

# Initialize dictionary to count matched categories
category_summary = defaultdict(int)

# Initialize a list to store paths of relevant documents
relevant_document_paths = []

# List all PDF files in the specified directory
document_paths = [os.path.join(docs_path, file) for file in os.listdir(docs_path) if file.endswith('.pdf')]

# Loop to filter relevant documents
for document_path in document_paths:
    matched_categories = []

    # Extract and clean text from the current document
    raw_text = extract_text_from_pdf(document_path)
    text = clean_text(raw_text)

    # Loop through each category and check for keyword matches
    for category, keywords in keywords_dict.items():
        for keyword in keywords:
            if fuzz.partial_ratio(keyword.lower(), text.lower()) >= SIMILARITY_THRESHOLD:
                matched_categories.append(category)
                category_summary[category] += 1  # Increment the count for the matched category
                break  # Stop checking more keywords if one has matched

    # Add to relevant document paths if it has matched categories
    if matched_categories:
        relevant_document_paths.append(document_path)
        print(f"Relevant Document: {os.path.basename(document_path)}")
        print(f"Matched Categories: {', '.join(set(matched_categories))}\n")

# Output the summary of category matches
print("Category Summary:")
for category, count in category_summary.items():
    print(f"{category}: {count} matches")


Relevant Document: Policing, Security and Community Safety Act 2024.pdf
Matched Categories: Financial and Economic Law

Relevant Document: Coroners (Amendment) Act 2024.pdf
Matched Categories: Employment Contracts and Agreements

Relevant Document: Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf
Matched Categories: Data and Privacy Law

Relevant Document: Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf
Matched Categories: Social Welfare Law, Employment Law Compliance

Relevant Document: Gas (Amendment) and Miscellaneous Provisions Act 2024.pdf
Matched Categories: Employment and Labour Law

Relevant Document: Research and Innovation Act 2024.pdf
Matched Categories: Research and Innovation Law, Employment and Labour Law

Relevant Document: Future Ireland Fund and Infrastructure, Climate and Nature Fund Act 2024.pdf
Matched Categories: Financial and Economic Law

Relevant Document: Health (Assisted Human Reproduction

In [16]:
from collections import defaultdict
import os
import re
import fitz  # PyMuPDF
from fuzzywuzzy import fuzz

# Ensure keywords_dict and SIMILARITY_THRESHOLD are defined as before

# Initialize category_summary
category_summary = defaultdict(int)

# Define the path to your documents
docs_path = r'D:\Data Analysis ML AI\irish_law'

# Function to extract text from each PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Function to clean text
def clean_text(text):
    text = re.sub(r'\n\d+\n', ' ', text)  # Remove page numbers
    text = re.sub(r'\s+', ' ', text)       # Remove extra whitespace
    text = re.sub(r'HEADER_TEXT|FOOTER_TEXT', '', text)  # Remove common header/footer text patterns
    non_legal_phrases = ["Confidential", "Draft Copy"]
    for phrase in non_legal_phrases:
        text = text.replace(phrase, '')
    return text.strip()

# List all PDF files in the specified directory
document_paths = [os.path.join(docs_path, file) for file in os.listdir(docs_path) if file.endswith('.pdf')]

# Loop to process each document
for document_path in document_paths:
    # Initialize matched_categories for each document
    matched_categories = []

    # Extract and clean text from the current document
    raw_text = extract_text_from_pdf(document_path)
    text = clean_text(raw_text)

    # Tokenize the text into words for the sliding window approach
    words_in_text = text.split()

    # Loop through each category and check for keyword matches
    for category, keywords in keywords_dict.items():
        category_matched = False  # Flag to indicate if the category has been matched
        for keyword in keywords:
            keyword_length = len(keyword.split())
            # Slide through the text with a window of keyword length
            for i in range(len(words_in_text) - keyword_length + 1):
                phrase = " ".join(words_in_text[i:i + keyword_length])
                if fuzz.partial_ratio(keyword.lower(), phrase.lower()) >= SIMILARITY_THRESHOLD:
                    if category not in matched_categories:
                        matched_categories.append(category)
                        category_summary[category] += 1  # Increment the count for the matched category
                    category_matched = True  # Set the flag to True since we've matched the category
                    break  # Stop checking more phrases if a match is found
            if category_matched:
                break  # Exit the keyword loop since the category has been matched

    # Output the matched categories for the document
    print(f"Document: {os.path.basename(document_path)}")
    if matched_categories:
        print(f"Matched Categories: {', '.join(matched_categories)}\n")
    else:
        print("No matching categories found.\n")

# Summary section for document matches by category
print("\nSummary of Document Matches by Category:")
for category, count in category_summary.items():
    print(f"{category}: {count} document(s) matched")


Document: Policing, Security and Community Safety Act 2024.pdf
Matched Categories: Employment Law, Employee Rights and Benefits, Labour Relations and Unions, Workplace Policies, Employment Contracts and Agreements, Employment Law Compliance, Financial and Economic Law, Technology and Digital Law, Social Welfare Law, Research and Innovation Law, Retirement and Pensions Law, Education and Training Law, Corporate and Business Law, Employment and Labour Law, Data and Privacy Law

Document: Digital Services Act 2024.pdf
Matched Categories: Employment Law, Employee Rights and Benefits, Labour Relations and Unions, Workplace Policies, Employment Contracts and Agreements, Employment Law Compliance, Financial and Economic Law, Technology and Digital Law, Social Welfare Law, Research and Innovation Law, Retirement and Pensions Law, Education and Training Law, Corporate and Business Law, Employment and Labour Law, Data and Privacy Law

Document: Finance (State Guarantees, International Financial 

In [17]:
# Summary section for document matches by category

print("\nSummary of Document Matches by Category:")
for category, count in category_summary.items():
    print(f"{category}: {count} document(s) matched")



Summary of Document Matches by Category:
Employment Law: 560 document(s) matched
Employee Rights and Benefits: 560 document(s) matched
Labour Relations and Unions: 483 document(s) matched
Workplace Policies: 560 document(s) matched
Employment Contracts and Agreements: 560 document(s) matched
Employment Law Compliance: 493 document(s) matched
Financial and Economic Law: 560 document(s) matched
Technology and Digital Law: 560 document(s) matched
Social Welfare Law: 560 document(s) matched
Research and Innovation Law: 560 document(s) matched
Retirement and Pensions Law: 560 document(s) matched
Education and Training Law: 560 document(s) matched
Corporate and Business Law: 560 document(s) matched
Employment and Labour Law: 560 document(s) matched
Data and Privacy Law: 560 document(s) matched


In [18]:
!pip install sentence-transformers torch pandas fitz fuzzywuzzy


  Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata (816 bytes)
Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl (20 kB)


Error processing line 1 of C:\Users\holme\anaconda3\Lib\site-packages\zope.interface-5.4.0-py3.12-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 206, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 810, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [19]:
import os
import re
import fitz  # PyMuPDF
from fuzzywuzzy import fuzz

# Step 1: Define keywords_dict at the start of the script
keywords_dict = {
    "Employment Law": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal",
        "Employment", "Redundancies", "Companies", "Labour", "Workplace", "Wages", "Union", "Employee", 
        "Contract", "Pensions", "Rights", "Protection", "Compensation", "Officer", "Board", "Agreement"
    ],
    "Employee Rights and Benefits": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal"
    ],
    "Labour Relations and Unions": [
        "Collective Bargaining", "Strike", "Industrial Action", "Union Membership", "Labor Dispute", 
        "Union Representation", "Employee Representation"
    ],
    "Workplace Policies": [
        "Harassment", "Discrimination", "Equal Opportunity", "Health and Safety", "Workplace Conduct", 
        "Professional Standards", "Ethics in Employment", "Grievance Procedures"
    ],
    "Employment Contracts and Agreements": [
        "Fixed-Term", "Permanent", "Temporary", "Probationary Period", "Notice Period", "Employment Contract", 
        "Part-Time", "Full-Time", "Independent Contractor"
    ],
    "Employment Law Compliance": [
        "Statutory Requirements", "Regulatory Compliance", "Employment Tribunal", "Disciplinary Action", 
        "Legal Protection", "Welfare", "Fair Treatment"
    ],
    "Financial and Economic Law": [
        "Finance", "Tax", "Economic", "Guarantees", "Funds", "Investment", "Banking", "Money", "Budget", 
        "Revenue", "Assets", "Financial", "Capital", "Insurance", "Fiscal"
    ],
    "Technology and Digital Law": [
        "Digital", "Data", "Privacy", "Cybersecurity", "Online", "Internet", "Information", "Communications", 
        "Telecommunications"
    ],
    "Social Welfare Law": [
        "Social Welfare", "Assistance", "Housing", "Disability", "Unemployment", "Pension", "Family Support", 
        "Public Assistance", "Benefits", "Community"
    ],
    "Research and Innovation Law": [
        "Research", "Innovation", "Agency", "Science", "Development", "Superannuation", "Board", "Funding", 
        "Minister", "Establishment"
    ],
    "Retirement and Pensions Law": [
        "Retirement", "Pension", "Savings", "Enrolment", "Superannuation"
    ],
    "Education and Training Law": [
        "Education", "Training", "Schools", "Teaching", "Apprenticeship", "Learning"
    ],
    "Corporate and Business Law": [
        "Corporate", "Business", "Trade", "Commerce", "Industry", "Company", "Commercial"
    ],
    "Employment and Labour Law": [
        "Employment", "Union", "Agreement", "Officer", "Publications", "Board", "Member", "Wages", "Labour", 
        "Contract", "Workforce", "Employee Benefits", "Labour Standards", "Industrial Relations", "Working Time", 
        "Paternity Leave", "Employee Rights", "Standards in Public Office"
    ],
    "Data and Privacy Law": [
        "Information", "Publications", "Notice", "Statutory", "Scheme", "Regulations", "Directive", 
        "Confidentiality", "Data Retention", "Information Security", "Disclosure", "Data Protection", 
        "Freedom of Information", "Protected Disclosures", "Privacy", "Official Secrets"
    ]
}

# Define other constants and variables
SIMILARITY_THRESHOLD = 80
docs_path = r'D:\Data Analysis ML AI\irish_law'

# Step 2: Function to extract text from each PDF document
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Step 3: Batch processing function
def batch_process_documents(doc_paths, keywords_dict):
    category_summary = {category: 0 for category in keywords_dict}

    for document_path in doc_paths:
        matched_categories = []
        document_text = extract_text_from_pdf(document_path)

        for category, keywords in keywords_dict.items():
            if any(fuzz.partial_ratio(keyword, document_text) >= SIMILARITY_THRESHOLD for keyword in keywords):
                if category not in matched_categories:
                    matched_categories.append(category)
                    category_summary[category] += 1

        # Output results per document
        print(f"\nRelevant Document: {os.path.basename(document_path)}")
        print(f"Matched Categories: {', '.join(matched_categories) if matched_categories else 'None'}")

    # Summary section
    print("\nSummary of Document Matches by Category:")
    for category, count in category_summary.items():
        print(f"{category}: {count} document(s) matched")

# Step 4: Run batch processing
# Generate list of document paths
document_paths = [os.path.join(docs_path, filename) for filename in os.listdir(docs_path) if filename.endswith('.pdf')]

# Run the batch processing function
batch_process_documents(document_paths, keywords_dict)



Relevant Document: Policing, Security and Community Safety Act 2024.pdf
Matched Categories: Employment Law, Employment Contracts and Agreements, Employment Law Compliance, Social Welfare Law, Research and Innovation Law, Employment and Labour Law

Relevant Document: Digital Services Act 2024.pdf
Matched Categories: Technology and Digital Law

Relevant Document: Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf
Matched Categories: Financial and Economic Law, Technology and Digital Law

Relevant Document: Coroners (Amendment) Act 2024.pdf
Matched Categories: Employment Contracts and Agreements

Relevant Document: Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf
Matched Categories: Research and Innovation Law, Data and Privacy Law

Relevant Document: Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf
Matched Categories: Employment Law Compliance, Social Welfa

In [20]:
def process_batches(document_paths, batch_size=5):  # Start with 5
    for i in range(0, len(document_paths), batch_size):
        batch = document_paths[i:i + batch_size]
        batch_process_documents(batch, keywords_dict)  # Adjusted function call


In [21]:
from collections import defaultdict
from fuzzywuzzy import fuzz

# Initialize the category summary to store the number of matches per category
category_summary = defaultdict(int)

# Function to process each document batch
def batch_process_documents(batch, keywords_dict):
    for document_path in batch:
        # Initialize matched categories for each document
        matched_categories = []
        
        # Extract text from the document
        text = extract_text_from_pdf(document_path)
        
        # Check each category and its keywords
        for category, keywords in keywords_dict.items():
            # Check for keyword matches using fuzz.partial_ratio
            if any(fuzz.partial_ratio(keyword, text) >= SIMILARITY_THRESHOLD for keyword in keywords):
                # Add to matched categories if not already matched
                if category not in matched_categories:
                    matched_categories.append(category)
                    category_summary[category] += 1  # Increment match count in summary
        
        # Print results for each document processed in batch
        print(f"Document '{os.path.basename(document_path)}' matched categories: {matched_categories}")

# Final summary function to display the document matches by category after all batches
def display_summary():
    print("\nSummary of Document Matches by Category:")
    for category, count in category_summary.items():
        print(f"{category}: {count} document(s) matched")

# Example usage:
document_paths = [os.path.join(docs_path, filename) for filename in os.listdir(docs_path) if filename.endswith('.pdf')]
process_batches(document_paths, batch_size=5)  # Adjust batch size as needed
display_summary()  # Display final summary after all batches are processed


Document 'Policing, Security and Community Safety Act 2024.pdf' matched categories: ['Employment Law', 'Employment Contracts and Agreements', 'Employment Law Compliance', 'Social Welfare Law', 'Research and Innovation Law', 'Employment and Labour Law']
Document 'Digital Services Act 2024.pdf' matched categories: ['Technology and Digital Law']
Document 'Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf' matched categories: ['Financial and Economic Law', 'Technology and Digital Law']
Document 'Coroners (Amendment) Act 2024.pdf' matched categories: ['Employment Contracts and Agreements']
Document 'Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf' matched categories: ['Research and Innovation Law', 'Data and Privacy Law']
Document 'Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf' matched categories: ['Employment Law Compliance', 'Social Welfare Law']
Documen

In [22]:
print("Document Paths Loaded:", document_paths)
if not document_paths:
    print("No document paths found. Check the directory or file paths.")


Document Paths Loaded: ['D:\\Data Analysis ML AI\\irish_law\\Policing, Security and Community Safety Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Digital Services Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Coroners (Amendment) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Local Government (Mayor of Limerick) and Miscellaneous Provisions Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Criminal Justice (Engagement of Children in Criminal Activity) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\European Arrest Warrant (Amendment) Act 2024.pdf', 'D:\\Data Analysis ML A

In [23]:
# Define the categories and keywords (if not already defined)
import os
import re
import fitz  # PyMuPDF
from fuzzywuzzy import fuzz
from collections import defaultdict, Counter

# Define constants and variables
SIMILARITY_THRESHOLD = 80
docs_path = r'D:\Data Analysis ML AI\irish_law'

# Define keywords_dict (only once)
keywords_dict = {
    "Employment Law": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal",
        "Employment", "Redundancies", "Companies", "Labour", "Workplace", "Wages", "Union", "Employee", 
        "Contract", "Pensions", "Rights", "Protection", "Compensation", "Officer", "Board", "Agreement"
    ],
    "Employee Rights and Benefits": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal"
    ],
    "Labour Relations and Unions": [
        "Collective Bargaining", "Strike", "Industrial Action", "Union Membership", "Labor Dispute", 
        "Union Representation", "Employee Representation"
    ],
    "Workplace Policies": [
        "Harassment", "Discrimination", "Equal Opportunity", "Health and Safety", "Workplace Conduct", 
        "Professional Standards", "Ethics in Employment", "Grievance Procedures"
    ],
    "Employment Contracts and Agreements": [
        "Fixed-Term", "Permanent", "Temporary", "Probationary Period", "Notice Period", "Employment Contract", 
        "Part-Time", "Full-Time", "Independent Contractor"
    ],
    "Employment Law Compliance": [
        "Statutory Requirements", "Regulatory Compliance", "Employment Tribunal", "Disciplinary Action", 
        "Legal Protection", "Welfare", "Fair Treatment"
    ],
    "Financial and Economic Law": [
        "Finance", "Tax", "Economic", "Guarantees", "Funds", "Investment", "Banking", "Money", "Budget", 
        "Revenue", "Assets", "Financial", "Capital", "Insurance", "Fiscal"
    ],
    "Technology and Digital Law": [
        "Digital", "Data", "Privacy", "Cybersecurity", "Online", "Internet", "Information", "Communications", 
        "Telecommunications"
    ],
    "Social Welfare Law": [
        "Social Welfare", "Assistance", "Housing", "Disability", "Unemployment", "Pension", "Family Support", 
        "Public Assistance", "Benefits", "Community"
    ],
    "Research and Innovation Law": [
        "Research", "Innovation", "Agency", "Science", "Development", "Superannuation", "Board", "Funding", 
        "Minister", "Establishment"
    ],
    "Retirement and Pensions Law": [
        "Retirement", "Pension", "Savings", "Enrolment", "Superannuation"
    ],
    "Education and Training Law": [
        "Education", "Training", "Schools", "Teaching", "Apprenticeship", "Learning"
    ],
    "Corporate and Business Law": [
        "Corporate", "Business", "Trade", "Commerce", "Industry", "Company", "Commercial"
    ],
    "Employment and Labour Law": [
        "Employment", "Union", "Agreement", "Officer", "Publications", "Board", "Member", "Wages", "Labour", 
        "Contract", "Workforce", "Employee Benefits", "Labour Standards", "Industrial Relations", "Working Time", 
        "Paternity Leave", "Employee Rights", "Standards in Public Office"
    ],
    "Data and Privacy Law": [
        "Information", "Publications", "Notice", "Statutory", "Scheme", "Regulations", "Directive", 
        "Confidentiality", "Data Retention", "Information Security", "Disclosure", "Data Protection", 
        "Freedom of Information", "Protected Disclosures", "Privacy", "Official Secrets"
    ]
}

# Function to extract text from each PDF document
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Function to clean the text
def clean_text(text):
    # Remove page numbers and extra whitespace
    text = re.sub(r'\n\d+\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # Remove headers, footers, and other non-informative phrases
    text = re.sub(r'HEADER_TEXT|FOOTER_TEXT', '', text)
    non_legal_phrases = ["Confidential", "Draft Copy"]
    for phrase in non_legal_phrases:
        text = text.replace(phrase, '')
    return text.strip()

# Initialize the category summary to store the number of matches per category
category_summary = defaultdict(int)

# Function to process each document batch
def batch_process_documents(batch, keywords_dict):
    for document_path in batch:
        # Initialize matched categories for each document
        matched_categories = []
        
        # Extract and clean text from the document
        text = extract_text_from_pdf(document_path)
        text = clean_text(text)
        
        # Check each category and its keywords using direct substring matching
        for category, keywords in keywords_dict.items():
            if any(keyword.lower() in text.lower() for keyword in keywords):
                if category not in matched_categories:
                    matched_categories.append(category)
                    category_summary[category] += 1  # Increment match count in summary
        
        # Print results for each document processed in batch
        print(f"Document '{os.path.basename(document_path)}' matched categories: {matched_categories}")

# Function to process documents in batches
def process_batches(document_paths, batch_size, keywords_dict):
    for i in range(0, len(document_paths), batch_size):
        batch = document_paths[i:i+batch_size]
        batch_process_documents(batch, keywords_dict)

# Final summary function to display the document matches by category after all batches
def display_summary():
    print("\nSummary of Document Matches by Category:")
    for category, count in category_summary.items():
        print(f"{category}: {count} document(s) matched")

# Main execution
document_paths = [os.path.join(docs_path, filename) for filename in os.listdir(docs_path) if filename.endswith('.pdf')]

print("Document Paths Loaded:", document_paths)
if not document_paths:
    print("No document paths found. Check the directory or file paths.")
else:
    process_batches(document_paths, batch_size=5, keywords_dict=keywords_dict)
    display_summary()  # Display final summary after all batches are processed

    # Test text extraction on a single document
    sample_document = document_paths[0]
    sample_text = extract_text_from_pdf(sample_document)

    # Print out a sample of the extracted text to verify
    print(f"\nExtracted text from '{sample_document}':\n")
    print(sample_text[:500])  # Display the first 500 characters

    # Function to find and prioritize category matches
    def find_category_matches(text, keywords_dict, top_n=3):
        category_counts = Counter()
        for category, keywords in keywords_dict.items():
            for keyword in keywords:
                if keyword.lower() in text.lower():
                    category_counts[category] += 1  # Count occurrences of each keyword in each category

        # Get the top N categories by match count
        top_matches = category_counts.most_common(top_n)
        return [category for category, count in top_matches]

    # Test refined function on the sample text
    matched_categories = find_category_matches(sample_text, keywords_dict)

    print(f"Top matched categories for '{os.path.basename(sample_document)}': {matched_categories}")

Document Paths Loaded: ['D:\\Data Analysis ML AI\\irish_law\\Policing, Security and Community Safety Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Digital Services Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Coroners (Amendment) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Local Government (Mayor of Limerick) and Miscellaneous Provisions Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Criminal Justice (Engagement of Children in Criminal Activity) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\European Arrest Warrant (Amendment) Act 2024.pdf', 'D:\\Data Analysis ML A

In [24]:
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import os

# Define an extended list of stop words, including most frequent keywords
stop_words = {
    "the", "and", "of", "to", "in", "for", "with", "by", "as", "on", "that", "is", "or", "be", "this",
    "are", "at", "it", "from", "an", "may", "which", "has", "such", "not", "was", "have", "shall", "act",
    "section", "subsection", "part", "regulation", "provision", "law", "title", "contents", "general",
    "amendment", "thereof", "pursuant", "hereby", "herein", "therein", "order", "minister", "prescribed",
    "includes", "provided", "means", "applies", "further", "within", "under", "applicable", "amended",
    "inserted", "following", "effective", "authority", "schedule", "accordance", "respect", "relevant",
    "any", "every", "each", "other", "been", "where", "but", "all", "upon", "into", "make", "period",
    "certain", "only", "deemed", "case", "established", "whereas", "subject", "etc", "including", "among",
    "through", "same", "time", "person", "persons", "those", "these", "out", "given", "new", "being",
    "2024", "minister", "regulations", "paragraph", "2024", "act", "shall", "section", "minister", "regulations", "regulation", "person", "paragraph", "subsection",
    "the", "and", "of", "to", "in", "for", "with", "by", "as", "on", "that", "is", "or", "be", "this",
    "are", "at", "it", "from", "an", "may", "which", "has", "such", "not", "was", "have", "shall", "act",
    "section", "subsection", "part", "regulation", "provision", "law", "title", "contents", "general",
    "amendment", "thereof", "pursuant", "hereby", "herein", "therein", "order", "minister", "prescribed",
    "includes", "provided", "means", "applies", "further", "within", "under", "applicable", "amended",
    "inserted", "following", "effective", "authority", "schedule", "accordance", "respect", "relevant",
    "any", "every", "each", "other", "been", "where", "but", "all", "upon", "into", "make", "period",
    "certain", "only", "deemed", "case", "established", "whereas", "subject", "etc", "including", "among",
    "through", "same", "time", "person", "persons", "those", "these", "out", "given", "new", "being"
}

# Clean text by removing stop words
def remove_stop_words(text):
    return " ".join([word for word in text.split() if word.lower() not in stop_words])

# Revised match_categories function to include stop word filtering
def match_categories(text, keywords_dict):
    category_matches = defaultdict(int)
    
    # Remove stop words from the text
    cleaned_text = remove_stop_words(text)
    
    for category, keywords in keywords_dict.items():
        for keyword in keywords:
            if fuzz.partial_ratio(keyword, cleaned_text) >= SIMILARITY_THRESHOLD:
                category_matches[category] += 1  # Increment match count

    # Sort categories by match count and return the top categories
    sorted_categories = sorted(category_matches, key=category_matches.get, reverse=True)
    top_categories = sorted_categories[:3]  # Return the top 3 categories
    return top_categories

# Revised process_batches function
def process_batches(document_paths, keywords_dict, batch_size=5):
    all_results = []
    
    for i in range(0, len(document_paths), batch_size):
        batch = document_paths[i:i + batch_size]
        for document_path in batch:
            try:
                print(f"\nProcessing document: {document_path}")
                extracted_text = extract_text_from_pdf(document_path)
                cleaned_text = remove_stop_words(extracted_text)
                top_categories = match_categories(cleaned_text, keywords_dict)
                
                if top_categories:
                    all_results.append({'Document': document_path, 'Top Categories': top_categories})
                    print(f"Top matched categories for '{document_path}': {top_categories}")
                else:
                    print(f"No relevant categories matched for '{document_path}'")
            except Exception as e:
                print(f"Error processing document '{document_path}': {e}")

    return all_results

# Run the batch processing with stop word filtering
all_document_results = process_batches(document_paths, keywords_dict)

# Summary report
print("\nSummary of Top Matched Categories for Each Document:")
for result in all_document_results:
    print(f"{result['Document']}: {result['Top Categories']}")


Processing document: D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf
Top matched categories for 'D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf': ['Employment Law', 'Employment Contracts and Agreements', 'Employment Law Compliance']

Processing document: D:\Data Analysis ML AI\irish_law\Digital Services Act 2024.pdf
Top matched categories for 'D:\Data Analysis ML AI\irish_law\Digital Services Act 2024.pdf': ['Technology and Digital Law']

Processing document: D:\Data Analysis ML AI\irish_law\Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf
Top matched categories for 'D:\Data Analysis ML AI\irish_law\Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf': ['Financial and Economic Law', 'Employment Law', 'Technology and Digital Law']

Processing document: D:\Data Analysis ML AI\irish_law\Coro

In [25]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\holme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
import nltk
nltk.data.path.append("C:\\Users\\holme\\AppData\\Roaming\\nltk_data")


In [27]:
import re
from transformers import GPT2TokenizerFast
from collections import defaultdict
from fuzzywuzzy import fuzz  # Ensure this package is installed with pip install fuzzywuzzy

# Initialize the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")  # Token limit compatibility

# Set OpenAI API token limit
MAX_TOKENS = 4096

# Define similarity threshold for matching
SIMILARITY_THRESHOLD = 70 

# Define a list of employment-specific keywords for summarization
employment_keywords = [
    # List keywords here, as shown in your previous request
]

# Define the categories and keywords for matching
keywords_dict = {
    "Employment Law": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal",
        "Employment", "Redundancies", "Companies", "Labour", "Workplace", "Wages", "Union", "Employee", 
        "Contract", "Pensions", "Rights", "Protection", "Compensation", "Officer", "Board", "Agreement"
    ],
    "Employee Rights and Benefits": [
        "Leave", "Maternity", "Paternity", "Sick Leave", "Work Hours", "Minimum Wage", "Pension", "Retirement", 
        "Overtime", "Annual Leave", "Holiday Pay", "Compensation", "Severance", "Termination", "Dismissal"
    ],
    "Labour Relations and Unions": [
        "Collective Bargaining", "Strike", "Industrial Action", "Union Membership", "Labor Dispute", 
        "Union Representation", "Employee Representation"
    ],
    "Workplace Policies": [
        "Harassment", "Discrimination", "Equal Opportunity", "Health and Safety", "Workplace Conduct", 
        "Professional Standards", "Ethics in Employment", "Grievance Procedures"
    ],
    "Employment Contracts and Agreements": [
        "Fixed-Term", "Permanent", "Temporary", "Probationary Period", "Notice Period", "Employment Contract", 
        "Part-Time", "Full-Time", "Independent Contractor"
    ],
    "Employment Law Compliance": [
        "Statutory Requirements", "Regulatory Compliance", "Employment Tribunal", "Disciplinary Action", 
        "Legal Protection", "Welfare", "Fair Treatment"
    ],
    "Financial and Economic Law": [
        "Finance", "Tax", "Economic", "Guarantees", "Funds", "Investment", "Banking", "Money", "Budget", 
        "Revenue", "Assets", "Financial", "Capital", "Insurance", "Fiscal"
    ],
    "Technology and Digital Law": [
        "Digital", "Data", "Privacy", "Cybersecurity", "Online", "Internet", "Information", "Communications", 
        "Telecommunications"
    ],
    "Social Welfare Law": [
        "Social Welfare", "Assistance", "Housing", "Disability", "Unemployment", "Pension", "Family Support", 
        "Public Assistance", "Benefits", "Community"
    ],
    "Research and Innovation Law": [
        "Research", "Innovation", "Agency", "Science", "Development", "Superannuation", "Board", "Funding", 
        "Minister", "Establishment"
    ],
    "Retirement and Pensions Law": [
        "Retirement", "Pension", "Savings", "Enrolment", "Superannuation"
    ],
    "Education and Training Law": [
        "Education", "Training", "Schools", "Teaching", "Apprenticeship", "Learning"
    ],
    "Corporate and Business Law": [
        "Corporate", "Business", "Trade", "Commerce", "Industry", "Company", "Commercial"
    ],
    "Employment and Labour Law": [
        "Employment", "Union", "Agreement", "Officer", "Publications", "Board", "Member", "Wages", "Labour", 
        "Contract", "Workforce", "Employee Benefits", "Labour Standards", "Industrial Relations", "Working Time", 
        "Paternity Leave", "Employee Rights", "Standards in Public Office"
    ],
    "Data and Privacy Law": [
        "Information", "Publications", "Notice", "Statutory", "Scheme", "Regulations", "Directive", 
        "Confidentiality", "Data Retention", "Information Security", "Disclosure", "Data Protection", 
        "Freedom of Information", "Protected Disclosures", "Privacy", "Official Secrets"
    ]
}
# Define a list of employment-specific keywords for summarization
employment_keywords = [
    # General Employment Terms
    "employment", "employee", "employer", "worker", "labour", "workforce", "staff", "job", "position", "work", "workplace",
    
    # Rights and Protections
    "rights", "protection", "fair treatment", "equality", "anti-discrimination", "harassment", "bullying", "safety", "health", "well-being",
    
    # Contracts and Agreements
    "contract", "employment contract", "fixed-term", "permanent", "temporary", "full-time", "part-time", "probationary", "agreement", 
    "non-compete", "non-disclosure", "notice period", "termination", "dismissal",
    
    # Leave and Absences
    "leave", "annual leave", "holiday", "maternity leave", "paternity leave", "sick leave", "parental leave", "bereavement leave", 
    "unpaid leave", "paid leave", "absence", "time off",
    
    # Compensation and Benefits
    "wages", "salary", "pay", "compensation", "remuneration", "overtime", "bonus", "incentive", "allowance", "severance", "termination pay",
    "benefits", "health insurance", "life insurance", "disability insurance", "pension", "retirement", "savings plan",
    
    # Working Conditions
    "working hours", "shift", "overtime", "rest period", "break", "working conditions", "accommodation", "flexible work", "remote work",
    
    # Disciplinary and Grievance Procedures
    "disciplinary action", "misconduct", "grievance", "complaint", "disciplinary procedure", "termination", "dismissal", "redundancy", "layoff",
    "performance management", "warning", "suspension", "demotion",
    
    # Unions and Collective Bargaining
    "union", "collective bargaining", "industrial action", "strike", "representation", "union membership", "trade union", "labor dispute", 
    "collective agreement",
    
    # Compliance and Legal Standards
    "statutory requirements", "compliance", "employment tribunal", "legal protection", "welfare", "regulations", "standards", 
    "labor laws", "labor standards", "equality", "diversity",
    
    # Retirement and Pensions
    "retirement", "pension", "retirement savings", "retirement age", "superannuation", "retirement fund", "401k", "pension scheme",
    
    # Training and Development
    "training", "development", "apprenticeship", "mentorship", "professional development", "skills training", "career growth",
    
    # Redundancy and Severance
    "redundancy", "severance", "layoff", "termination", "dismissal", "end of contract", "redundancy pay", "notice", "final pay",
    
    # Data Privacy and Confidentiality
    "data privacy", "confidentiality", "data protection", "privacy policy", "data security", "confidential information",
    
    # Miscellaneous
    "performance review", "promotion", "employee benefits", "unemployment", "job description", "resignation", "work permit", "visa", "probation period"
]


In [50]:
import torch
print(torch.cuda.is_available())  # Should return True
# For CUDA 11.3
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113


True
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


Error processing line 1 of C:\Users\holme\anaconda3\Lib\site-packages\zope.interface-5.4.0-py3.12-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 206, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 810, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [52]:
!pip install transformers


Error processing line 1 of C:\Users\holme\anaconda3\Lib\site-packages\zope.interface-5.4.0-py3.12-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 206, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 810, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [53]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Initialize the summarization model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

C:\Users\holme\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\holme\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [56]:
sample_text = "This is a test sentence to check tokenizer initialization."
tokens = tokenizer.encode(sample_text)
decoded_text = tokenizer.decode(tokens)
print(f"Encoded Tokens: {tokens}")
print(f"Decoded Text: {decoded_text}")

Encoded Tokens: [0, 713, 16, 10, 1296, 3645, 7, 1649, 19233, 6315, 49164, 4, 2]
Decoded Text: <s>This is a test sentence to check tokenizer initialization.</s>


In [58]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Test the tokenizer
sample_text = "This is a test sentence to check tokenizer initialization."
tokens = tokenizer.encode(sample_text)
decoded_text = tokenizer.decode(tokens)

# Print results
print(f"Encoded Tokens: {tokens}")
print(f"Decoded Text: {decoded_text}")


Encoded Tokens: [0, 713, 16, 10, 1296, 3645, 7, 1649, 19233, 6315, 49164, 4, 2]
Decoded Text: <s>This is a test sentence to check tokenizer initialization.</s>


In [60]:
import os
import re
import fitz  # PyMuPDF
import nltk
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from fuzzywuzzy import fuzz

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize the summarization model and tokenizer
model_name = "facebook/bart-large-cnn"  # A good summarization model that supports GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Set token limit (depends on the model's max length)
MAX_TOKENS = 1024  # BART's maximum input length

# Define similarity threshold for matching
SIMILARITY_THRESHOLD = 70

# Define stop words
stop_words = set(stopwords.words('english'))

# Define the categories and keywords for matching
keywords_dict = {
    # ... [Your existing keywords_dict content]
}

# Define the employment-specific keywords for summarization
employment_keywords = [
    # ... [Your existing employment_keywords content]
]

# Function to extract text from each PDF document
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text")
    return text

# Function to clean the text
def clean_text(text):
    # Remove page numbers and extra whitespace
    text = re.sub(r'\n\d+\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # Remove headers, footers, and other non-informative phrases
    text = re.sub(r'HEADER_TEXT|FOOTER_TEXT', '', text, flags=re.IGNORECASE)
    # Remove stop words
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text.strip()

# Function to split text into chunks within the token limit
def split_text_into_chunks(text, max_tokens=MAX_TOKENS):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ''
    current_length = 0

    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens=False)
        sentence_length = len(tokenized_sentence)

        if current_length + sentence_length <= max_tokens:
            current_chunk += ' ' + sentence
            current_length += sentence_length
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_length = sentence_length

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Function to summarize text using the model
def summarize_text(text):
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=MAX_TOKENS, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Function to match categories using fuzzy matching
def match_categories(text, keywords_dict):
    category_matches = defaultdict(int)
    for category, keywords in keywords_dict.items():
        for keyword in keywords:
            if fuzz.partial_ratio(keyword.lower(), text.lower()) >= SIMILARITY_THRESHOLD:
                category_matches[category] += 1  # Increment match count
    # Sort categories by match count and return the top categories
    sorted_categories = sorted(category_matches, key=category_matches.get, reverse=True)
    top_categories = sorted_categories[:3]  # Return the top 3 categories
    return top_categories

# Main processing function
def process_documents(document_paths, keywords_dict):
    all_results = []

    for document_path in document_paths:
        try:
            print(f"\nProcessing document: {document_path}")
            # Extract and clean text
            raw_text = extract_text_from_pdf(document_path)
            cleaned_text = clean_text(raw_text)

            # Split text into chunks suitable for the model
            chunks = split_text_into_chunks(cleaned_text)

            # Summarize each chunk
            summaries = []
            for chunk in chunks:
                summary = summarize_text(chunk)
                summaries.append(summary)

            # Combine summaries
            full_summary = ' '.join(summaries)

            # Match categories
            top_categories = match_categories(full_summary, keywords_dict)

            # Store results
            result = {
                'Document': os.path.basename(document_path),
                'Top Categories': top_categories,
                'Summary': full_summary
            }
            all_results.append(result)

            # Output results
            print(f"Top matched categories for '{document_path}': {top_categories}")
            print(f"Summary:\n{full_summary}\n")

        except Exception as e:
            print(f"Error processing document '{document_path}': {e}")

    return all_results

# Example usage
if __name__ == "__main__":
    # Define the path to your documents
    docs_path = r'D:\Data Analysis ML AI\irish_law'  # Update this path to your directory

    # Generate list of document paths
    document_paths = [os.path.join(docs_path, filename) for filename in os.listdir(docs_path) if filename.endswith('.pdf')]

    print("Document Paths Loaded:", document_paths)
    if not document_paths:
        print("No document paths found. Check the directory or file paths.")
    else:
        # Process documents
        all_document_results = process_documents(document_paths, keywords_dict)

        # Summary report
        print("\nSummary of Top Matched Categories for Each Document:")
        for result in all_document_results:
            print(f"{result['Document']}: {result['Top Categories']}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\holme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\holme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using device: cuda


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Document Paths Loaded: ['D:\\Data Analysis ML AI\\irish_law\\Policing, Security and Community Safety Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Digital Services Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Coroners (Amendment) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Local Government (Mayor of Limerick) and Miscellaneous Provisions Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\Criminal Justice (Engagement of Children in Criminal Activity) Act 2024.pdf', 'D:\\Data Analysis ML AI\\irish_law\\European Arrest Warrant (Amendment) Act 2024.pdf', 'D:\\Data Analysis ML A

In [68]:
test_path = "D:\\Data Analysis ML AI\\irish_law\\sample_act.pdf"
test_text = extract_text_from_pdf(test_path)
print(f"Extracted Text for {test_path}:\n{test_text[:500]}")  # Print the first 500 characters


Extracted Text for D:\Data Analysis ML AI\irish_law\sample_act.pdf:
Number 20 of 2024
Automatic Enrolment Retirement Savings System Act 2024
Number 20 of 2024
AUTOMATIC ENROLMENT RETIREMENT SAVINGS SYSTEM ACT 2024
CONTENTS
PART 1
PRELIMINARY AND GENERAL
Section
1.  Short title and commencement
2.  Definitions
3.  Regulations
4.  Service of documents
5.  Expenses
PART 2
 AN TÚDARÁS NÁISIÚNTA UM UATHROLLÚ COIGILTIS SCOIR
CHAPTER 1
Establishment of An tÚdarás Náisiúnta um Uathrollú Coigiltis Scoir
6.  Definitions (Part 2)
7.  Establishment day
8.  An tÚdarás Náisiú


In [76]:
import os
import fitz  # PyMuPDF

def extract_text_from_document(file_path):
    """
    Extracts text from a PDF document using PyMuPDF (fitz).
    
    Args:
        file_path (str): Path to the PDF file.
    
    Returns:
        str: The extracted text from the document.
    """
    try:
        text = ""
        print(f"Opening document: {file_path}")
        
        # Open the PDF file
        with fitz.open(file_path) as doc:
            total_pages = doc.page_count
            print(f"Document '{file_path}' opened successfully. Total pages: {total_pages}")
            
            # Iterate through pages and extract text
            for page_num in range(total_pages):
                page = doc.load_page(page_num)
                page_text = page.get_text("text")  # Explicitly specify text extraction type
                text += page_text
                print(f"Processed page {page_num + 1} of {total_pages}")
        
        print(f"Text extraction completed successfully for: {file_path}")
        return text.strip()  # Return stripped text to remove trailing spaces or newlines
    
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

def process_all_pdfs_in_directory(directory_path, output_folder):
    """
    Processes all PDF files in the specified directory.
    
    Args:
        directory_path (str): Path to the directory containing PDF files.
        output_folder (str): Folder where extracted text files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):  # Only process PDF files
            pdf_path = os.path.join(directory_path, filename)
            print(f"Processing file: {pdf_path}")
            
            # Extract text
            text = extract_text_from_document(pdf_path)
            
            # Save extracted text to a file
            if text:
                output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(text)
                print(f"Extracted text saved to: {output_file}")
            else:
                print(f"No text extracted for {filename}")

# Example Usage
if __name__ == "__main__":
    pdf_directory = r"D:\Data Analysis ML AI\irish_law"  # Path to folder containing PDFs
    output_directory = r"D:\Data Analysis ML AI\irish_law_extracted"  # Path to save extracted text files
    
    process_all_pdfs_in_directory(pdf_directory, output_directory)


Processing file: D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf
Opening document: D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf
Document 'D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf' opened successfully. Total pages: 291
Processed page 1 of 291
Processed page 2 of 291
Processed page 3 of 291
Processed page 4 of 291
Processed page 5 of 291
Processed page 6 of 291
Processed page 7 of 291
Processed page 8 of 291
Processed page 9 of 291
Processed page 10 of 291
Processed page 11 of 291
Processed page 12 of 291
Processed page 13 of 291
Processed page 14 of 291
Processed page 15 of 291
Processed page 16 of 291
Processed page 17 of 291
Processed page 18 of 291
Processed page 19 of 291
Processed page 20 of 291
Processed page 21 of 291
Processed page 22 of 291
Processed page 23 of 291
Processed page 24 of 291
Processed page 25 of 291
Processed page 26 of 291
Processed page 27

In [78]:
test_path = "D:\\Data Analysis ML AI\\irish_law\\sample_act.pdf"
test_text = extract_text_from_document(test_path)
print(f"Extracted Text for {test_path}:\n{test_text[:500]}")  # Print the first 500 characters


Opening document: D:\Data Analysis ML AI\irish_law\sample_act.pdf
Document 'D:\Data Analysis ML AI\irish_law\sample_act.pdf' opened successfully. Total pages: 105
Processed page 1 of 105
Processed page 2 of 105
Processed page 3 of 105
Processed page 4 of 105
Processed page 5 of 105
Processed page 6 of 105
Processed page 7 of 105
Processed page 8 of 105
Processed page 9 of 105
Processed page 10 of 105
Processed page 11 of 105
Processed page 12 of 105
Processed page 13 of 105
Processed page 14 of 105
Processed page 15 of 105
Processed page 16 of 105
Processed page 17 of 105
Processed page 18 of 105
Processed page 19 of 105
Processed page 20 of 105
Processed page 21 of 105
Processed page 22 of 105
Processed page 23 of 105
Processed page 24 of 105
Processed page 25 of 105
Processed page 26 of 105
Processed page 27 of 105
Processed page 28 of 105
Processed page 29 of 105
Processed page 30 of 105
Processed page 31 of 105
Processed page 32 of 105
Processed page 33 of 105
Processed page 34 of 

In [79]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [80]:
import os

# Define the directory with your PDF files
directory_path = "D:\\Data Analysis ML AI\\irish_law"
document_paths = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path) if filename.endswith(".pdf")]
print(f"Total documents found: {len(document_paths)}")


Total documents found: 560


In [81]:
from collections import defaultdict

# Initialize dictionary to count matched categories
category_summary = defaultdict(int)

# Initialize a list to store paths of relevant documents
relevant_document_paths = []

# Define the path to your documents
docs_path = r'D:\Data Analysis ML AI\irish_law'

# List all PDF files in the specified directory
document_paths = [os.path.join(docs_path, file) for file in os.listdir(docs_path) if file.endswith('.pdf')]

# Loop to filter relevant documents
for document_path in document_paths:
    matched_categories = []

    # Extract text from the current document
    text = extract_text_from_document(document_path)
    
    # Loop through each category and check for keyword matches
    for category, keywords in keywords_dict.items():
        if any(fuzz.partial_ratio(keyword, text) >= SIMILARITY_THRESHOLD for keyword in keywords):
            matched_categories.append(category)
            category_summary[category] += 1  # Increment the count for the matched category
    
    # Add to relevant document paths if it has matched categories
    if matched_categories:
        relevant_document_paths.append(document_path)
        print(f"Relevant Document: {os.path.basename(document_path)}")
        print(f"Matched Categories: {', '.join(matched_categories)}\n")


Opening document: D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf
Document 'D:\Data Analysis ML AI\irish_law\Policing, Security and Community Safety Act 2024.pdf' opened successfully. Total pages: 291
Processed page 1 of 291
Processed page 2 of 291
Processed page 3 of 291
Processed page 4 of 291
Processed page 5 of 291
Processed page 6 of 291
Processed page 7 of 291
Processed page 8 of 291
Processed page 9 of 291
Processed page 10 of 291
Processed page 11 of 291
Processed page 12 of 291
Processed page 13 of 291
Processed page 14 of 291
Processed page 15 of 291
Processed page 16 of 291
Processed page 17 of 291
Processed page 18 of 291
Processed page 19 of 291
Processed page 20 of 291
Processed page 21 of 291
Processed page 22 of 291
Processed page 23 of 291
Processed page 24 of 291
Processed page 25 of 291
Processed page 26 of 291
Processed page 27 of 291
Processed page 28 of 291
Processed page 29 of 291
Processed page 30 of 291
Processed page 31 of

In [92]:
# Process only relevant documents in batches
all_document_results = process_batches(relevant_document_paths, keywords_dict)

# Print a summary for each processed document
for result in all_document_results:
    print(f"{result['Document']} - Top Categories: {result['Top Categories']}")
    print(f"Prepared Text for API (first 500 chars):\n{result['Prepared Text'][:500]}\n")


In [98]:
from nltk.corpus import stopwords

# Download NLTK stopwords
import nltk
nltk.download('stopwords')

# Get NLTK stopwords
nltk_stopwords = set(stopwords.words('english'))

# Your custom stopwords
custom_stopwords = {
    "2024", "act", "shall", "section", "minister", "regulations", "regulation", "person", 
    "paragraph", "subsection", "the", "and", "of", "to", "in", "for", "with", "by", "as", 
    "on", "that", "is", "or", "be", "this", "are", "at", "it", "from", "an", "may", "which", 
    "has", "such", "not", "was", "have", "shall", "part", "thereof", "where", "upon", "been", 
    "being", "those", "out", "any", "other", "all", "under", "into", "same", "each", "through", "the", "and", "of", "to", "in", "for", "with", "by", "as", "on", "that", "is", "or", "be", "this", 
    "are", "at", "it", "from", "an", "may", "which", "has", "such", "not", "was", "have", "shall", "act", 
    "section", "subsection", "part", "regulation", "provision", "law", "title", "contents", "general", 
    "amendment", "thereof", "pursuant", "hereby", "herein", "therein", "order", "minister", "prescribed", 
    "includes", "provided", "means", "applies", "further", "within", "under", "applicable", "amended", 
    "inserted", "following", "effective", "authority", "schedule", "accordance", "respect", "relevant", 
    "any", "every", "each", "other", "been", "where", "but", "all", "upon", "into", "make", "period", 
    "certain", "only", "deemed", "case", "established", "whereas", "subject", "etc", "including", "among", 
    "through", "same", "time", "person", "persons", "those", "these", "out", "given", "new", "being",
    
   
    "the", "and", "of", "to", "in", "for", "with", "by", "as", "on", "that", "is", "or", 
    "be", "this", "are", "at", "it", "from", "an", "may", "which", "has", "such", "not", 
    "was", "have", "shall", "part", "thereof", "where", "upon", "been", "being", "those", 
    "out", "any", "other", "all", "under", "into", "same", "each", "through",

   
    "act", "section", "subsection", "schedule", "amendment", "regulation", "provision", 
    "law", "article", "clause", "pursuant", "hereby", "wherein", "therein", "herein", 
    "order", "minister", "authority", "service", "further", "applicable", "prescribed", 
    "deemed", "case", "established", "subject", "effective", "include", "provide", "pursuant", 
    "enact", "respect", "inserted", "enacted", "establishment", "authority", "certain", 
    "matter", "apply", "authority", "hereof", "title", "statutory",
    

    "document", "number", "provided", "following", "order", "respect", "means", "includes",
    "effective", "date", "whereas", "accordance", "subject", "application", "new", "time",
    "make", "certain", "given", "prescribed", "considered", "included", "deemed", "issued",
    "case", "regulatory", "statutory", "requirement", "established", "specified", "made"
}
# Combine both stopword sets
combined_stopwords = nltk_stopwords.union(custom_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\holme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re

def clean_text(text):
    """
    Cleans the input text by:
    - Lowercasing
    - Removing punctuation
    - Removing numbers
    - Removing stopwords (custom + NLTK)
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove numbers
    
    # Tokenize and remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in combined_stopwords]
    
    # Return the cleaned text
    return ' '.join(filtered_words)



In [104]:
# Words that were removed
removed_words = [word for word in sample_text.split() if word.lower() not in cleaned_sample.split()]
print(f"Removed Words: {removed_words}")


Removed Words: ['The', 'Minister', 'that', 'the', 'to', 'this', 'Act', 'are']


In [109]:
import re
from fuzzywuzzy import fuzz

def clean_and_extract_relevant_sections(document_path, stopwords, keywords):
    """
    Extracts, cleans, and filters relevant sections of text from a document.
    
    Args:
        document_path (str): Path to the PDF document.
        stopwords (set): Set of stopwords to remove.
        keywords (list): List of keywords to filter relevant sections.

    Returns:
        str: Relevant and cleaned text from the document.
    """
    # Extract raw text from the document
    raw_text = extract_text_from_document(document_path)

    # Clean text
    cleaned_text = clean_text(raw_text, stopwords)

    # Filter relevant sections based on keywords
    relevant_sections = []
    for line in cleaned_text.split("\n"):
        if any(fuzz.partial_ratio(keyword.lower(), line.lower()) > 70 for keyword in keywords):
            relevant_sections.append(line)
    
    # Combine relevant sections
    return " ".join(relevant_sections)


In [111]:
def clean_text(text, stopwords):
    """
    Cleans the input text by:
    - Lowercasing
    - Removing punctuation
    - Removing numbers
    - Removing stopwords
    
    Args:
        text (str): Raw text to clean.
        stopwords (set): Set of stopwords to remove.
    
    Returns:
        str: Cleaned text.
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove numbers
    
    # Tokenize and filter stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    
    # Rejoin filtered words
    return ' '.join(filtered_words)


In [115]:
# Example Run
document_paths = ["D:\\Data Analysis ML AI\\irish_law\\sample_act.pdf"]  # Update with your document paths
stopwords = combined_stopwords  # Ensure stopwords are defined
employment_keywords = ["contract", "employee rights", "dismissal", "redundancy"]  # Define keywords

all_document_results = process_for_api(document_paths, stopwords, employment_keywords)

print("\nSummary of Prepared Text for Each Document:")
for result in all_document_results:
    print(f"{result['Document']}: Token count {len(tokenizer.encode(result['Prepared Text']))}")

# Extract and print raw text directly from the document
raw_text = extract_text_from_document("D:\\Data Analysis ML AI\\irish_law\\sample_act.pdf")
print(f"Extracted Text (First 500 characters):\n{raw_text[:500]}")
print(f"Text Length: {len(raw_text)}")

Processing batch 1/1

Processing document: D:\Data Analysis ML AI\irish_law\sample_act.pdf
Opening document: D:\Data Analysis ML AI\irish_law\sample_act.pdf
Document 'D:\Data Analysis ML AI\irish_law\sample_act.pdf' opened successfully. Total pages: 105
Processed page 1 of 105
Processed page 2 of 105
Processed page 3 of 105
Processed page 4 of 105
Processed page 5 of 105
Processed page 6 of 105
Processed page 7 of 105
Processed page 8 of 105
Processed page 9 of 105
Processed page 10 of 105
Processed page 11 of 105
Processed page 12 of 105
Processed page 13 of 105
Processed page 14 of 105
Processed page 15 of 105
Processed page 16 of 105
Processed page 17 of 105
Processed page 18 of 105
Processed page 19 of 105
Processed page 20 of 105
Processed page 21 of 105
Processed page 22 of 105
Processed page 23 of 105
Processed page 24 of 105
Processed page 25 of 105
Processed page 26 of 105
Processed page 27 of 105
Processed page 28 of 105
Processed page 29 of 105
Processed page 30 of 105
Proce

In [125]:
from PyPDF2 import PdfReader

def extract_text(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


In [131]:
import os
from transformers import GPT2Tokenizer
from PyPDF2 import PdfReader  # For text PDFs. Use OCR for scanned PDFs if needed.

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define the directory containing all PDF files
pdf_directory = r"D:\Data Analysis ML AI\irish_law"

# Generate a list of all PDF file paths in the directory
document_paths = [os.path.join(pdf_directory, file) for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

# Define stopwords and keywords
stopwords = set(["the", "and", "is", "in", "to", "of", "for", "with", "on", "by", "an", "at"])  # Define stopwords
employment_keywords = ["contract", "employee rights", "dismissal", "redundancy"]

# Function to extract text from a PDF
def extract_text(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""  # Handle cases where no text is extracted
        return text.strip()
    except Exception as e:
        print(f"Error extracting text from {file_path}: {e}")
        return ""

# Function to summarize text (placeholder for actual summarization logic)
def summarize_text(text):
    # Replace with your summarization model or API
    return text[:500] + "..." if len(text) > 500 else text  # Dummy summarization for demonstration

# Function to split text into chunks
def split_text(text, max_token_limit):
    tokens = tokenizer.encode(text)
    return [tokenizer.decode(tokens[i:i + max_token_limit]) for i in range(0, len(tokens), max_token_limit)]

# Function to process a single document
def process_single_document(file_path, stopwords, max_token_limit=1024):
    try:
        # Extract text from the PDF
        text = extract_text(file_path)

        # Check if text is extracted
        if not text:
            print(f"Warning: No text extracted from {file_path}.")
            return {"Document": os.path.basename(file_path), "Prepared Text": "", "Token Count": 0}

        # Clean text by removing stopwords
        cleaned_text = " ".join([word for word in text.split() if word.lower() not in stopwords])

        # Tokenize and check token count
        token_count = len(tokenizer.encode(cleaned_text))
        if token_count > max_token_limit:
            # Split text into manageable chunks
            chunks = split_text(cleaned_text, max_token_limit)
            cleaned_text = " ".join(chunks)
            token_count = len(tokenizer.encode(cleaned_text))

        return {
            "Document": os.path.basename(file_path),
            "Prepared Text": cleaned_text,
            "Token Count": token_count,
        }
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return {"Document": os.path.basename(file_path), "Prepared Text": "", "Token Count": 0}

# Process all PDF files
all_document_results = []
for file_path in document_paths:
    result = process_single_document(file_path, stopwords)
    all_document_results.append(result)

# Print a summary of the results
print("\nSummary of Prepared Text for Each Document:")
for result in all_document_results:
    print(f"{result['Document']}: Token count {result['Token Count']}")

# Identify and retry problematic files
problematic_files = [result["Document"] for result in all_document_results if result["Token Count"] == 0]
if problematic_files:
    print("\nProblematic files identified for retry:")
    for file in problematic_files:
        print(file)
    print("\nRetrying problematic files...")
    for file in problematic_files:
        file_path = os.path.join(pdf_directory, file)
        result = process_single_document(file_path, stopwords)
        print(f"Retry result for {file}: Token count {result['Token Count']}")

# Optional: Summarize large documents
max_token_limit = 1024  # Define the token limit for your model
for result in all_document_results:
    if result["Token Count"] > max_token_limit:
        segments = split_text(result["Prepared Text"], max_token_limit)
        summarized_text = []
        for segment in segments:
            # Apply summarization
            summary = summarize_text(segment)
            summarized_text.append(summary)
        result["Prepared Text"] = " ".join(summarized_text)
        result["Token Count"] = len(tokenizer.encode(result["Prepared Text"]))

print("\nSummarization completed for large documents.")


Token indices sequence length is longer than the specified maximum sequence length for this model (116966 > 1024). Running this sequence through the model will result in indexing errors



Summary of Prepared Text for Each Document:
Policing, Security and Community Safety Act 2024.pdf: Token count 117047
Digital Services Act 2024.pdf: Token count 32092
Finance (State Guarantees, International Financial Institution Funds and Miscellaneous Provisions) Act 2024.pdf: Token count 34328
Coroners (Amendment) Act 2024.pdf: Token count 4602
Human Tissue (Transplantation, Post-Mortem, Anatomical Examination and Public Display) Act 2024.pdf: Token count 52169
Social Welfare and Civil Law (Miscellaneous Provisions) Act 2024.pdf: Token count 2380
Local Government (Mayor of Limerick) and Miscellaneous Provisions Act 2024.pdf: Token count 70437
Criminal Justice (Engagement of Children in Criminal Activity) Act 2024.pdf: Token count 687
European Arrest Warrant (Amendment) Act 2024.pdf: Token count 8911
Road Traffic Act 2024.pdf: Token count 3676
Gas (Amendment) and Miscellaneous Provisions Act 2024.pdf: Token count 8616
Court Proceedings (Delays) Act 2024.pdf: Token count 9978
Health (

In [133]:
import pandas as pd

# Convert results to a DataFrame
df = pd.DataFrame(all_document_results)

# Display the first few rows of the DataFrame
print(df.head())

# Check for problematic files and token distribution
print("\nProblematic Files (Zero Token Count):")
print(df[df['Token Count'] == 0])

print("\nToken Count Summary:")
print(df['Token Count'].describe())

# Save to a CSV file if needed
df.to_csv("processed_documents_summary.csv", index=False)


                                            Document  \
0  Policing, Security and Community Safety Act 20...   
1                      Digital Services Act 2024.pdf   
2  Finance (State Guarantees, International Finan...   
3                  Coroners (Amendment) Act 2024.pdf   
4  Human Tissue (Transplantation, Post-Mortem, An...   

                                       Prepared Text  Token Count  
0  Number 1 2024 Policing, Security Community Saf...        12737  
1  Number 2 2024 Digital Services Act 2024Number ...         3478  
2  Number 3 2024 Finance (State Guarantees, Inter...         3496  
3  Number 4 2024 Coroners (Amendment) Act 2024Num...          562  
4  Number 5 2024 Human Tissue (Transplantation, P...         5444  

Problematic Files (Zero Token Count):
Empty DataFrame
Columns: [Document, Prepared Text, Token Count]
Index: []

Token Count Summary:
count      560.000000
mean       815.758929
std       1215.985239
min        145.000000
25%        374.500000
50%       

In [135]:
df.to_csv("processed_documents_summary.csv", index=False)


In [172]:
!pip install gensim
!pip install numpy scipy


  Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl (24.0 MB)
Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl (45.9 MB)


Error processing line 1 of C:\Users\holme\anaconda3\Lib\site-packages\zope.interface-5.4.0-py3.12-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 206, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 810, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.2.post1 requires joblib>=0.14, which is not installed.
pmdarima 2.0.4 requires joblib>=0.11, which is not installed.
scikit-learn 1.5.2 requires joblib>=1.2.0, which is not installed.
scikit-learn 1.5.2 requires threadpoolctl>=3.1.0, which is not installed.
tsfresh 0.20.3 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


Error processing line 1 of C:\Users\holme\anaconda3\Lib\site-packages\zope.interface-5.4.0-py3.12-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 206, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 810, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [173]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Load your dataset
import pandas as pd
df = pd.read_csv("processed_documents_summary.csv")

# Preprocess text data
documents = df['Prepared Text'].dropna().values
tokenized_documents = [doc.split() for doc in documents]  # Tokenize each document

# Create a dictionary and corpus
dictionary = corpora.Dictionary(tokenized_documents)
corpus = [dictionary.doc2bow(text) for text in tokenized_documents]

# Train LDA model
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42)  # Adjust num_topics

# Display topics
topics = lda.print_topics(num_words=10)
print("Extracted Topics and Keywords:")
for topic_id, topic in topics:
    print(f"Topic {topic_id + 1}: {topic}")

# Assign dominant topic to each document
dominant_topics = []
for bow in corpus:
    topic_probs = lda.get_document_topics(bow)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    dominant_topics.append(f"Topic {dominant_topic + 1}")

# Add dominant topic to DataFrame
df['Dominant Topic'] = dominant_topics

# Save the results
df.to_csv("document_topics_summary_gensim.csv", index=False)

print("\nDocument Topic Assignments:")
print(df[['Document', 'Dominant Topic']])


Extracted Topics and Keywords:
Topic 1: 0.016*"or" + 0.015*"a" + 0.013*"2024" + 0.010*"as" + 0.009*"section" + 0.009*"be" + 0.008*"Act" + 0.008*"shall" + 0.007*"that" + 0.007*"No."
Topic 2: 0.014*"No." + 0.014*"2024" + 0.014*"or" + 0.012*"Act" + 0.011*"a" + 0.010*"Regulation" + 0.009*"be" + 0.008*"(EU)" + 0.007*"L" + 0.007*"OJ"
Topic 3: 0.022*"a" + 0.022*"or" + 0.012*"be" + 0.009*"Act" + 0.009*"section" + 0.007*"as" + 0.007*"that" + 0.006*"not" + 0.006*"No." + 0.005*"under"
Topic 4: 0.019*"a" + 0.017*"or" + 0.011*"be" + 0.010*"as" + 0.010*"section" + 0.009*"2024" + 0.008*"shall" + 0.008*"under" + 0.007*"Act" + 0.006*"that"
Topic 5: 0.016*"a" + 0.014*"or" + 0.012*"Act" + 0.011*"be" + 0.010*"section" + 0.009*"2024" + 0.009*"under" + 0.008*"as" + 0.008*"No." + 0.007*"shall"

Document Topic Assignments:
                                              Document Dominant Topic
0    Policing, Security and Community Safety Act 20...        Topic 4
1                        Digital Services Act 202

In [199]:
import scipy
import greenlet
print("Both packages are successfully installed and working!")


Both packages are successfully installed and working!


In [203]:
import pyLDAvis
import pyLDAvis.gensim_models
import gensim
from gensim import corpora

pyLDAvis.enable_notebook()  # Optional for notebooks
vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
pyLDAvis.show(vis_data)


ModuleNotFoundError: No module named 'sklearn.manifold'